Teaching the Agent a New Trick on-the-Fly
=========================
Here we show agent one example from its own README file, then ask the agent code up and activate a new command for itself on-the-fly, then ask it to use the new command to gather data

In [1]:
import os
from easy_llm_agents.commands import BaseCommand, handlers
conv = BaseCommand.create_conversation(
    'gpt-4',
    metadata={'work_dir': os.path.expanduser('~/easy_llm_agents/examples/playground')},  # Save output files for prosperity
    model_options={'temperature': 0.8, 'max_tokens': 600}    # creative mode
) 
driver = BaseCommand.get_driver(conv, messenger=handlers.print_messages, overseer=handlers.do_nothing, qa=handlers.do_nothing)

### First show the example and ask it to code up the new command


In [2]:
print(driver.send("""You are in Github repo `easy_llm_agents` of user `virtualzx-nad`.
Read the section 'How to implement a new command' verbatim from `README.md` and implement command to get weather forecast for the next 5 days.
If you choose to use an API, make sure you read the doc to understand the parameters and returns structure, and think about what need to be passed and returned for you command.
I cannot provide you with API keys due to privacy restrictions, but the code can directly retrieve the API keys from `self.metadata`. 
Do not use APIs that are not free, and tell me what API keys need to be passed into metadata.  
Note the code, then verify that modules are lazily imported in methods rather than at the beginning of the script before submit the code.
The command will be created or updated as soon as you submit the code, but make sure to notify me and get approval before you start using them.
"""))

<Message from search>
info: Googling GitHub repo easy_llm_agents by user virtualzx-nad
<Message from read_page>
info: Extract and summarize https://github.com/virtualzx-nad/easy_llm_agents/blob/main/README.md with instruction `Read the section 'How to implement a new command' verbatim from README.md`
<Message from read_page>
info: Total lines: 629. Tokens: 2134. Summarizations~ 2
<Message from read_page>
info: Performing summarization for line #1 to 586 with approx 1999 tokens in text
<Message from read_page>
info: Performing summarization for line #587 to 629 with approx 100 tokens in text
<Message from self_note>
thoughts: ['Find a suitable free API for weather forecast', "Create a new command using the provided example from 'How to implement a new command'", 'Add necessary API keys to metadata', 'Ensure modules are lazily imported in methods']
<Message from search>
info: Googling free weather forecast API
<Message from self_note>
thoughts: ['I will use OpenWeatherMap API as it provi

### Add API keys per its request
Add the requested API key to metadata.  Then check if the new command is now in command list


In [3]:
import os
conv.metadata['openweathermap_api_key'] = os.environ['OPENWEATHER_API_KEY']
print(BaseCommand.generate_command_list())

Your responses must be a list of commands, expressed as a Python list of dicts.  Each dict correspond to a command with the following fields
        - command:  name of the command
        - summary:  one sentence summary of the purpose of invoking the command
        - content:  content that is passed to the worker, usually a list.  Each command should define what need to be provided in the content.
    Information requested will be returned in next prompt.  If a command does not produce the expected effect, take a note to yourself about why do you think that happened, and make sure you try a different approach instead of keep repeating a failed one.
    Do not add any explanations outside of the list, do not enclose it in quotation, and speak to the user only through commands.
    The full list of valid commands are:
  - `answer`: Answer a question or report that a requested task has been successfully performed, when you are confident of the results.
 - `self_note`: Note to yourself 

### Now ask it to run the new command
We told it to not run it yet because I need to go register for an API key first;  Now it is in metadata we can tell it to continue.
Then ask the question and see it uses the command

In [4]:
print(driver.send("""OK I have added the API key to the metadata.  
Now you can invoke command `weather_forecast`. 
What's the weather in San Francisco the day after tomorrow?"""))

<Message from self_note>
thoughts: ["Update the implementation of weather_forecast command to correctly use lazy import for 'requests'", 'Test the updated implementation']
<Message from python>
action: install_package
package: requests
<Message from python>
info: Executing code snippet
code: 
from easy_llm_agents.commands import BaseCommand

class WeatherForecastCommand(BaseCommand, command='weather_forecast', description='Get weather forecast for the next 5 days. The following should be supplied in content:\n\n- `city`: name of the city\n\n- `country`: two-letter country code'):
    
    def generate_prompt(self):
        # Lazy import requests module
        import requests
        
        url = 'https://api.openweathermap.org/data/2.5/forecast'
        params = {
            'q': f'{self.content[0]["city"]},{self.content[0]["country"]}',
            'units': 'metric',
            'appid': self.metadata["openweathermap_api_key"],
            'cnt': 5
        }
        
        resp 

### Bugs discovered!

When it runs the code it didn't run.  This is fairly common for GPT generated code.  So it tries again, then again. 
Third time was the charm though!  And it was able to report the weather.  It's gonna get cold!

### Under the hood

Here are the full language chain including the tooling part

In [5]:
for entry in conv.history:
    print(f"{entry['role']}: {entry['content']}")

user: 
user instruction: You are in Github repo `easy_llm_agents` of user `virtualzx-nad`.
Read the section 'How to implement a new command' verbatim from `README.md` and implement command to get weather forecast for the next 5 days.
If you choose to use an API, make sure you read the doc to understand the parameters and returns structure, and think about what need to be passed and returned for you command.
I cannot provide you with API keys due to privacy restrictions, but the code can directly retrieve the API keys from `self.metadata`. 
Do not use APIs that are not free, and tell me what API keys need to be passed into metadata.  
Note the code, then verify that modules are lazily imported in methods rather than at the beginning of the script before submit the code.
The command will be created or updated as soon as you submit the code, but make sure to notify me and get approval before you start using them.

assistant: [
  {
    "command": "search",
    "summary": "Find the GitHub r